In [1]:
import os
os.chdir('../movies')
from movieLens import MovieLens

#ml = MovieLens()

In [2]:
# from surprise import AlgoBase
# from surprise import PredictionImpossible

import math
import numpy as np
import heapq

# Cosine similarity metric (genres)
def getGenreSimilarityScore(movieId_1, movieId_2, genres):
    genres1 = genres[movieId_1]
    genres2 = genres[movieId_2]

    sumxx,sumxy,sumyy = 0,0,0

    for i in range(len(genres1)):
        x = genres1[i]
        y = genres2[i]

        sumxx += x*x
        sumyy += y*y
        sumxy += x*y

    res =sumxy/math.sqrt(sumxx*sumyy)
    return res


# Exponential decay function
def getYearSimilarityScore(year_movie1, year_movie2):
    if year_movie1 == None or year_movie2 == None:
        res = 0
    else:
        dif = abs(int(year_movie1)-int(year_movie2))
        res = math.exp(-dif / 10.0)
    return res

Content based recommender of movies based on the genre and the year of the movies

In [3]:
# KNN
def getNeighbors(movie_title):

    # Load up genre vectors for every movie
    ml = MovieLens()
    genres = ml.getGenres()

    neighbors = []
    #movie_title = "Toy Story"
    movie_df = ml.movies[ml.movies['title'] == movie_title]
    filtered_df = ml.movies[ml.movies['title'] != movie_title]

    for index, row in filtered_df.iterrows():
        #print(row['movieId'])
        #print(index)
        genre_score = getGenreSimilarityScore(movie_df['movieId'].values[0],row['movieId'],genres)
        #year_score = getYearSimilarityScore(movie_df['year'].values[0],row['year'])

        #Compute the global score
        score = genre_score #* year_score
        #print(score)

        # Add the result
        neighbors.append((score,row['movieId']))

    # Extract the top-K most-similar ratings based on the similarity score)
    # Returns a list of tuples(score,movieId)
    top = heapq.nlargest(15, neighbors, key=lambda t: t[0])
    
    top_movies = []
    
    for item in top:
        title = ml.movieID_to_name.get(item[1])
        #print(title)
        top_movies.append(title)
    
    # Returns the list with the title of the top k recommendations
    return top_movies


#     # Compute average sim score of K neighbors weighted by user ratings
#     simTotal = weightedSum = 0
#     for (simScore, rating) in k_neighbors:
#         if (simScore > 0):
#             simTotal += simScore
#             weightedSum += simScore * rating

#     if (simTotal == 0):
#         raise PredictionImpossible('No neighbors')

#     predictedRating = weightedSum / simTotal

#     return predictedRating

In [4]:
top = getNeighbors("Toy Story")
top

['Antz',
 'Toy Story 2',
 'Adventures of Rocky and Bullwinkle, The',
 "Emperor's New Groove, The",
 'Monsters, Inc.',
 'Wild, The',
 'Shrek the Third',
 'Tale of Despereaux, The',
 'Asterix and the Vikings (Astérix et les Vikings)',
 'Turbo',
 'The Good Dinosaur',
 'Moana',
 'Space Jam',
 'Shrek',
 "Twelve Tasks of Asterix, The (Les douze travaux d'Astérix)"]